# Machine learning to predict age from rs-fmri

## Load the data

In [ ]:
%matplotlib inline

In [ ]:
# Lets fetch the data!
from nilearn import datasets
data = datasets.fetch_development_fmri()

In [ ]:
len(data.func)

## Extract features

### Retrieve the atlas for extracting features and an example subject

In [ ]:
parcellations = datasets.fetch_atlas_basc_multiscale_2015(version='sym')
atlas_filename = parcellations.scale064

print('Atlas ROIs are located in nifti image (4D) at: %s' %
       atlas_filename)

In [ ]:
from nilearn import plotting

plotting.plot_roi(atlas_filename, draw_cross=False)

In [ ]:
fmri_filenames = data.func[0]
print(fmri_filenames)

In [ ]:
from nilearn import image 

averaged_Img = image.mean_img(fmri_filenames)
plotting.plot_stat_map(averaged_Img)

### Extract signals on a parcellation defined by labels

In [ ]:
from nilearn.input_data import NiftiLabelsMasker

masker = NiftiLabelsMasker(labels_img=atlas_filename, 
                           standardize=True, 
                           memory='nilearn_cache', 
                           verbose=1)

# Here we go from nifti files to the signal time series in a 
# numpy array. Note how we give confounds to be regressed out 
# during signal extraction
conf = data.confounds[0]
time_series = masker.fit_transform(fmri_filenames, confounds=conf)

In [ ]:
type(time_series)

In [ ]:
time_series.shape

In [ ]:
import pandas
conf_df = pandas.read_csv(conf,sep='\t')
conf_df.head()

In [ ]:
conf_df.shape

### Compute and display a correlation matrix

In [ ]:
from nilearn.connectome import ConnectivityMeasure

correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrix = correlation_measure.fit_transform([time_series])[0]
correlation_matrix.shape
# Exercise 5
# The numbers representing the shape of the connectivity matrix (64 x 64) represent the regions of interest (ROIs)

In [ ]:
import numpy as np
# Mask the main diagonal for visualization:
np.fill_diagonal(correlation_matrix, 0)

# The labels we have start with the background (0), hence we skip the
# first label  
plotting.plot_matrix(correlation_matrix, figure=(10, 8),   
                     labels=range(time_series.shape[-1]),
                     vmax=0.8, vmin=-0.8, reorder=False)

# matrices are ordered for block-like representation

### Extract features from the whole dataset

In [ ]:
# Here is a really simple for loop

for i in range(10):
    print('the number is', i)

In [ ]:
container = []
for i in range(10):
    container.append(i)

container

In [ ]:
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
from nilearn import datasets

# load atlas
multiscale = datasets.fetch_atlas_basc_multiscale_2015()
atlas_filename = multiscale.scale064

# initialize masker (change verbosity)
masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True, 
                           memory='nilearn_cache', verbose=0)

# initialize correlation measure, set to vectorize
correlation_measure = ConnectivityMeasure(kind='correlation', vectorize=True,
                                         discard_diagonal=True)

In [ ]:
all_features = [] # here is where we will put the data (a container)

for i,sub in enumerate(data.func):
    # extract the timeseries from the ROIs in the atlas
    time_series = masker.fit_transform(sub, confounds=data.confounds[i])
    # create a region x region correlation matrix
    correlation_matrix = correlation_measure.fit_transform([time_series])[0]
    # add to our container
    all_features.append(correlation_matrix)
    # keep track of status
    print('finished %s of %s'%(i+1,len(data.func)))

In [ ]:
# Let's save the data to disk
import numpy as np

#np.savez_compressed('MAIN_BASC064_subsamp_features',a = all_features)

In [ ]:
feat_file = 'MAIN_BASC064_subsamp_features.npz'
X_features = np.load(feat_file)['a']

In [ ]:
X_features.shape
# Exercise 5
# The features we are using in the model are the shape of the time series data and the connectivity matrix derives from fMRI data. 
# The numbers representing the feature matrix (155, 2016) are 155 samples (subjects or sessions) and 2016 features. 

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(X_features, aspect='auto')
plt.colorbar()
plt.title('feature matrix')
plt.xlabel('features')
plt.ylabel('subjects')

## Get Y (our target) and assess its distribution

In [ ]:
# Let's load the phenotype data
import pandas

pheno = pandas.DataFrame(data.phenotypic)
pheno.head()

In [ ]:
y_age = pheno['Age']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.distplot(y_age)

## Prepare data for machine learning

In [ ]:
age_class = pheno['AgeGroup']
age_class.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

# Split the sample to training/validation with a 60/40 ratio, and 
# stratify by age class, and also shuffle the data.

X_train, X_val, y_train, y_val = train_test_split(
                                                    X_features, # x
                                                    y_age, # y
                                                    test_size = 0.4, # 60%/40% split  
                                                    shuffle = True, # shuffle dataset
                                                                    # before splitting
                                                    stratify = age_class,  # keep
                                                                           # distribution
                                                                           # of ageclass
                                                                           # consistent
                                                                           # betw. train
                                                                           # & test sets.
                                                    random_state = 123 # same shuffle each
                                                                       # time
                                                                       )

# print the size of our training and test groups
print('training:', len(X_train),
     'testing:', len(X_val))

In [ ]:
sns.distplot(y_train,label='train')
sns.distplot(y_val,label='test')
plt.legend()

## Run your first model!

In [ ]:
from sklearn.svm import SVR
l_svr = SVR(kernel='linear') # define the model

l_svr.fit(X_train, y_train) # fit the model

In [ ]:

# predict the training data based on the model
y_pred = l_svr.predict(X_train) 

# caluclate the model accuracy
acc = l_svr.score(X_train, y_train) 


In [ ]:
# print results
print('accuracy (R2)', acc)

sns.regplot(x=y_pred,y=y_train)
plt.xlabel('Predicted Age')

In [ ]:
from sklearn.model_selection import train_test_split

# Split the sample to training/test with a 75/25 ratio, and 
# stratify by age class, and also shuffle the data.

age_class2 = pheno.loc[y_train.index,'AgeGroup']

X_train2, X_test, y_train2, y_test = train_test_split(
                                                    X_train, # x
                                                    y_train, # y
                                                    test_size = 0.25, # 75%/25% split  
                                                    shuffle = True, # shuffle dataset
                                                                    # before splitting
                                                    stratify = age_class2,  # keep
                                                                           # distribution
                                                                           # of ageclass
                                                                           # consistent
                                                                           # betw. train
                                                                           # & test sets.
                                                    random_state = 123 # same shuffle each
                                                                       # time
                                                                       )

# print the size of our training and test groups
print('training:', len(X_train2),
     'testing:', len(X_test))

In [ ]:
from sklearn.metrics import mean_absolute_error

# fit model just to training data
l_svr.fit(X_train2,y_train2)

# predict the *test* data based on the model trained on X_train2
y_pred = l_svr.predict(X_test) 

# caluclate the model accuracy
acc = l_svr.score(X_test, y_test) 
mae = mean_absolute_error(y_true=y_test,y_pred=y_pred)

In [ ]:
# print results
print('accuracy (R2) = ', acc)
print('MAE = ',mae)

sns.regplot(x=y_pred,y=y_test)
plt.xlabel('Predicted Age')

In [ ]:
from sklearn.model_selection import cross_val_predict, cross_val_score

# predict
y_pred = cross_val_predict(l_svr, X_train, y_train, cv=10)
# scores
acc = cross_val_score(l_svr, X_train, y_train, cv=10)
mae = cross_val_score(l_svr, X_train, y_train, cv=10, scoring='neg_mean_absolute_error')

In [ ]:
for i in range(10):
    print('Fold {} -- Acc = {}, MAE = {}'.format(i, acc[i],-mae[i]))

In [ ]:
from sklearn.metrics import r2_score

overall_acc = r2_score(y_train, y_pred)
overall_mae = mean_absolute_error(y_train,y_pred)
print('R2:',overall_acc)
print('MAE:',overall_mae)

sns.regplot(x=y_pred, y=y_train)
plt.xlabel('Predicted Age')

## Tweak your model

### Normalize the target data

In [ ]:
# Create a log transformer function and log transform Y (age)

from sklearn.preprocessing import FunctionTransformer

log_transformer = FunctionTransformer(func = np.log, validate=True)
log_transformer.fit(y_train.values.reshape(-1,1))
y_train_log = log_transformer.transform(y_train.values.reshape(-1,1))[:,0]

sns.distplot(y_train_log)
plt.title("Log-Transformed Age")

In [ ]:
# re-intialize the model
l_svr = SVR(kernel='linear') 

# predict
y_pred = cross_val_predict(l_svr, X_train, y_train_log, cv=10)

# scores
acc = r2_score(y_train_log, y_pred)
mae = mean_absolute_error(y_train_log,y_pred)

print('R2:',acc)
print('MAE:',mae)

sns.regplot(x=y_pred, y=y_train_log)
plt.xlabel('Predicted Log Age')
plt.ylabel('Log Age')

### Tweak the hyperparameters

In [ ]:
SVR?

In [ ]:
from sklearn.model_selection import validation_curve

C_range = 10. ** np.arange(-3, 8) # A range of different values for C

train_scores, valid_scores = validation_curve(l_svr, X_train, y_train_log, 
                                              param_name= "C",
                                              param_range = C_range,
                                              cv=10,
                                             scoring='neg_mean_squared_error')


In [ ]:
# A bit of pandas magic to prepare the data for a seaborn plot

tScores = pandas.DataFrame(train_scores).stack().reset_index()
tScores.columns = ['C','Fold','Score']
tScores.loc[:,'Type'] = ['Train' for x in range(len(tScores))]

vScores = pandas.DataFrame(valid_scores).stack().reset_index()
vScores.columns = ['C','Fold','Score']
vScores.loc[:,'Type'] = ['Validate' for x in range(len(vScores))]

ValCurves = pandas.concat([tScores,vScores]).reset_index(drop=True)
ValCurves.head()

In [ ]:
# And plot!

g = sns.catplot(x='C',y='Score',hue='Type',data=ValCurves,kind='point')
plt.xticks(range(10))
g.set_xticklabels(C_range, rotation=90)

In [ ]:
from sklearn.model_selection import GridSearchCV

C_range = 10. ** np.arange(-3, 8)
epsilon_range = 10. ** np.arange(-3, 8)

param_grid = dict(epsilon=epsilon_range, C=C_range)

grid = GridSearchCV(l_svr, param_grid=param_grid, cv=10)

grid.fit(X_train, y_train_log)

In [ ]:
print(grid.best_params_)

In [ ]:
y_pred = cross_val_predict(SVR(kernel='linear',
                               C=grid.best_params_['C'],
                               epsilon=grid.best_params_['epsilon'], 
                               gamma='auto'), 
                           X_train, y_train_log, cv=10)

# scores
acc = r2_score(y_train_log, y_pred)
mae = mean_absolute_error(y_train_log,y_pred)

print('R2:',acc)
print('MAE:',mae)

sns.regplot(x=y_pred, y=y_train_log)
plt.xlabel('Predicted Log Age')
plt.ylabel('Log Age')

### Trying a more complicated model

In [ ]:
validation_curve?

In [ ]:
from sklearn.model_selection import validation_curve

degree_range = list(range(1,8)) # A range of different values for C

train_scores, valid_scores = validation_curve(SVR(kernel='poly',
                                                  gamma='scale'
                                                 ), 
                                              X=X_train, y=y_train_log, 
                                              param_name= "degree",
                                              param_range = degree_range,
                                              cv=10,
                                             scoring='neg_mean_squared_error')


In [ ]:
# A bit of pandas magic to prepare the data for a seaborn plot

tScores = pandas.DataFrame(train_scores).stack().reset_index()
tScores.columns = ['Degree','Fold','Score']
tScores.loc[:,'Type'] = ['Train' for x in range(len(tScores))]

vScores = pandas.DataFrame(valid_scores).stack().reset_index()
vScores.columns = ['Degree','Fold','Score']
vScores.loc[:,'Type'] = ['Validate' for x in range(len(vScores))]

ValCurves = pandas.concat([tScores,vScores]).reset_index(drop=True)
ValCurves.head()

In [ ]:
# And plot!

# g = sns.catplot(x='Degree',y='Score',hue='Type',data=ValCurves,kind='point')
# plt.xticks(range(7))
# g.set_xticklabels(degree_range, rotation=90)

g = sns.catplot(x='Degree', y='Score', hue='Type', data=ValCurves, kind='point')
plt.xticks(range(len(degree_range)), degree_range)
g.set_xticklabels(degree_range, rotation=90)
plt.title('Validation Curve for Polynomial SVR')
plt.ylabel('Negative Mean Squared Error')
plt.xlabel('Polynomial Degree')
plt.show()

# Exercise 6
# Increasing the complexity of the models, as shown by the polynomial degrees, can lead to overfitting where the train error decreases, but the validation error increases.
# This indicates that the model is fitting the noise in the training data rather than the underlying pattern.

### Feature selection

In [ ]:
from sklearn.feature_selection import SelectPercentile, f_regression
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline

# # Build a tiny pipeline that does feature selection (top 20% of features), 
# # and then prediction with our linear svr model.
# model = Pipeline([
#     ('feature_selection',SelectPercentile(f_regression,percentile=20)),
#     ('prediction', l_svr)
#                  ])

# Exercise 3
from sklearn.decomposition import PCA
model = Pipeline([
    ('feature_selection',PCA(n_components=0.9)),
    ('prediction', l_svr)
                 ])

y_pred = [] # a container to catch the predictions from each fold
y_index = [] # just in case, the index for each prediction

# # First we create 10 splits of the data
# skf = KFold(n_splits=10, shuffle=True, random_state=123)

# # For each split, assemble the train and test samples 
# for tr_ind, te_ind in skf.split(X_train):
#     X_tr = X_train[tr_ind]
#     y_tr = y_train_log[tr_ind]
#     X_te = X_train[te_ind]
#     y_index += list(te_ind) # store the index of samples to predict
    
#     # and run our pipeline 
#     model.fit(X_tr, y_tr) # fit the data to the model using our mini pipeline
#     predictions = model.predict(X_te).tolist() # get the predictions for this fold
#     y_pred += predictions # add them to the list of predictions

# Exercise 4
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()

# For each split, assemble the train and test samples 
for tr_ind, te_ind in loo.split(X_train):
    X_tr = X_train[tr_ind]
    y_tr = y_train_log[tr_ind]
    X_te = X_train[te_ind]
    y_index += list(te_ind) # store the index of samples to predict
    
    # and run our pipeline 
    model.fit(X_tr, y_tr) # fit the data to the model using our mini pipeline
    predictions = model.predict(X_te).tolist() # get the predictions for this fold
    y_pred += predictions # add them to the list of predictions

In [ ]:
acc = r2_score(y_train_log[y_index], y_pred)
mae = mean_absolute_error(y_train_log[y_index],y_pred)

print('R2:',acc)
print('MAE:',mae)

sns.regplot(x=np.array(y_pred), y=y_train_log[y_index])
plt.xlabel('Predicted Log Age')
plt.ylabel('Log Age')

## Can our model predict age in completely un-seen data?

In [ ]:
# Notice how we use the Scaler that was fit to X_train and apply to X_test,
# rather than creating a new Scaler for X_test
y_val_log = log_transformer.transform(y_val.values.reshape(-1,1))[:,0]

In [ ]:
# Exercise 6
from sklearn.svm import LinearSVR
l_svr = LinearSVR(C=1.0, max_iter=10000)  # The C parameter controls the regularization strength
# l_svr = SVR(kernel='linear') # define the model
l_svr.fit(X_train, y_train_log) # fit to training data
y_pred = l_svr.predict(X_val) # classify age class using testing data
acc = l_svr.score(X_val, y_val_log) # get accuracy (r2)
mae = mean_absolute_error(y_val_log, y_pred) # get mae

# print results
print('accuracy (r2) =', acc)
print('mae = ',mae)

# plot results
sns.regplot(x=y_pred, y=y_val_log)
plt.xlabel('Predicted Log Age')
plt.ylabel('Log Age')

In [ ]:
# # Assignment
# from sklearn.model_selection import train_test_split

# # To store results of multiple splits
# all_accuracies = []
# all_maes = []

# # Repeat the train-validation split 10 times
# for i in range(10):
#     # Create a new train-validation split
#     X_train, X_val, y_train, y_val = train_test_split(
#         X_features, y_age, test_size=0.4, shuffle=True, stratify=age_class, random_state=i)
    
#     # Transform the target variable using the log transformation
#     y_train_log = log_transformer.transform(y_train.values.reshape(-1,1))[:,0]
#     y_val_log = log_transformer.transform(y_val.values.reshape(-1,1))[:,0]

#     # Fit the model on the training data
#     l_svr.fit(X_train, y_train_log)
    
#     # Predict the ages on the validation data
#     y_pred = l_svr.predict(X_val)
    
#     # Calculate accuracy (R2 score) and mean absolute error (MAE)
#     acc = l_svr.score(X_val, y_val_log)
#     mae = mean_absolute_error(y_val_log, y_pred)
    
#     # Store the results
#     all_accuracies.append(acc)
#     all_maes.append(mae)
    
#     # Print the results for each split
#     print(f'Split {i+1}: accuracy (R2) = {acc}, MAE = {mae}')

# # Plot the range of predictions
# plt.figure(figsize=(10, 6))
# sns.boxplot(data=[all_accuracies, all_maes], orient='h')
# plt.yticks([0, 1], ['R2 Score', 'MAE'])
# plt.title('Range of Predictions Across 10 Splits')
# plt.show()

### Interpreting model feature importances

In [ ]:
l_svr.coef_

In [ ]:
plt.bar(range(l_svr.coef_.shape[-1]),l_svr.coef_[0])
plt.title('feature importances')
plt.xlabel('feature')
plt.ylabel('weight')

In [ ]:
correlation_measure.inverse_transform(l_svr.coef_).shape

In [ ]:
from nilearn import plotting

feat_exp_matrix = correlation_measure.inverse_transform(l_svr.coef_)[0]

plotting.plot_matrix(feat_exp_matrix, figure=(10, 8),  
                     labels=range(feat_exp_matrix.shape[0]),
                     reorder=False,
                    tri='lower')

In [ ]:
coords = plotting.find_parcellation_cut_coords(atlas_filename)

In [ ]:
plotting.plot_connectome(feat_exp_matrix, coords, colorbar=True)

In [ ]:
plotting.plot_connectome(feat_exp_matrix, coords, colorbar=True, edge_threshold=0.035)

In [ ]:
plotting.view_connectome(feat_exp_matrix, coords, edge_threshold='98%',
                        edge_cmap='viridis')